# Simple Radar

In [1]:
import pamtra2
import collections
import refractiveIndex
import numpy as np
import pandas as pn
import xarray as xr
from collections import OrderedDict
from copy import deepcopy, copy
import toolz
import collections
import matplotlib.pyplot as plt

try:
    %matplotlib inline
except:
    pass

In [2]:
import pyPamtraRadarSimulator



# create Profile

In [3]:
additionalDims = collections.OrderedDict()

nHeights = 3

pam2 = pamtra2.pamtra2(
    nLayer=nHeights,
    hydrometeors=['cloud'],
    additionalDims = additionalDims,
    frequencies = [3e9],
)


In [4]:
pam2.profile

<xarray.customProfile>
Dimensions:              (frequency: 1, hydrometeor: 1, layer: 3)
Coordinates:
  * layer                (layer) int64 0 1 2
  * hydrometeor          (hydrometeor) <U5 'cloud'
  * frequency            (frequency) float64 3e+09
Data variables:
    height               (layer) float64 nan nan nan
    temperature          (layer) float64 nan nan nan
    pressure             (layer) float64 nan nan nan
    relativeHumidity     (layer) float64 nan nan nan
    horizontalWind       (layer) float64 nan nan nan
    verticalWind         (layer) float64 nan nan nan
    eddyDissipationRate  (layer) float64 nan nan nan
    waterContent         (layer, hydrometeor) float64 nan nan nan
    wavelength           (frequency) float64 0.09993

Fill with some dummie values

In [5]:

pam2.profile.height[:] = 1000
pam2.profile.temperature[:] = 250 
pam2.profile.relativeHumidity[:] = 90
pam2.profile.pressure[:] = 100000
pam2.profile.eddyDissipationRate[:] = 1e-3
pam2.profile.horizontalWind[:] = 10



For testing, dask arrays are turned on by chunking.

In [6]:
pam2.profile = pam2.profile.chunk({'layer':1, 'frequency':1})


pam2.profile 
pam2.addMissingVariables()

<xarray.customProfile>
Dimensions:              (frequency: 1, hydrometeor: 1, layer: 3)
Coordinates:
  * layer                (layer) int64 0 1 2
  * hydrometeor          (hydrometeor) <U5 'cloud'
  * frequency            (frequency) float64 3e+09
Data variables:
    height               (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    temperature          (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    pressure             (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    relativeHumidity     (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    horizontalWind       (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    verticalWind         (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    eddyDissipationRate  (layer) float64 dask.array<shape=(3,), chunksize=(1,)>
    waterContent         (layer, hydrometeor) float64 dask.array<shape=(3, 1), chunksize=(1, 1)>
    wavelength           (frequency) float64 dask.array<shape=(1,), chunksize=

In [7]:
# # OR EASIER if desired: 
# pam2 = pamtra2.importers.profiles.usStandardAtmosphere(heigths)

# Describe Hydrometeors
To describe the hydrometeors, you can use functions, xr.DataArrays or scalars. Teh functions must support xr.DataArrays. The kwargs of the functions are gathered from (in this order)

1. pam2.hydrometors.name.profile variables (name must match). E.g. density (after it has been calculated) 
2. pam2.profile variables (name must match). E.g. temperature
3. Other provided kwargs (e.g. Dmin, DMax, N0 etc)
4. Default function arguments. Can be turned of with useFuncArgDefaults = False to make sure everything is explicitly covered.



In [8]:

pam2.describeHydrometeor(
    pamtra2.hydrometeors.softEllipsoidFixedDensity,
    name = 'cloud', #or None, then str(index)
    nBins =2,
    sizeBounds = pamtra2.hydrometeors.size.linspaceBounds, 
    sizeCenter = pamtra2.hydrometeors.size.boundsToMid,
    sizeBoundsWidth = pamtra2.hydrometeors.size.boundsWidth,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.monoDisperse, 
    aspectRatio = 1.0,
    mass = pamtra2.hydrometeors.mass.ellipsoid,
    density = pamtra2.hydrometeors.density.water,
    crossSectionArea = pamtra2.hydrometeors.crossSectionArea.sphere,
    relativePermittivity = refractiveIndex.water.turner_kneifel_cadeddu, #replace with refractiveIndex.water.Turner.n
    scattering = pamtra2.hydrometeors.scattering.Rayleigh,
    fallVelocity = pamtra2.hydrometeors.fallVelocity.khvorostyanov01_drops,
    Dmin  = 0.001 - .5e-10,
    Dmax  = 0.001 + .5e-10,
    Ntot =  xr.DataArray(np.logspace(-2,2,nHeights),coords=[pam2.profile.layer]),
#     Ntot = 10,
    useFuncArgDefaults = False,
)


sizeBounds <function linspaceBounds at 0x11ada4c80>
callable
sizeCenter <function boundsToMid at 0x11ada4d90>
callable
sizeBoundsWidth <function boundsWidth at 0x11ada4e18>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x31bd61f28>
callable
crossSectionArea <function sphere at 0x31bd69620>
callable
sizeDistribution <function monoDisperse at 0x11ada4ea0>
callable
relativePermittivity <function turner_kneifel_cadeddu at 0x41c995400>
callable
scattering <function Rayleigh at 0x31bd69a60>
callable
fallVelocity <function khvorostyanov01_drops at 0x41c998158>
callable
nBins 2
not callable 2
Dmin 0.00099999995
not callable 0.00099999995
Dmax 0.00100000005
not callable 0.00100000005
Ntot <xarray.DataArray (layer: 3)>
array([1.e-02, 1.e+00, 1.e+02])
Coordinates:
  * layer    (layer) int64 0 1 2
not callable <xarray.DataArray (layer: 3)>
array([1.e-02, 1.e+00, 1.e+02])
Coordinates:
  * layer    (layer) int64 0 1 2


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping


The profile arrays have been populated, but they contain daks arrays. These are only placeholders, they have not been calculated yet!

In [9]:
pam2.hydrometeors.cloud.profile

<xarray.Dataset>
Dimensions:                  (frequency: 1, layer: 3, sizeBin: 2, sizeBin1: 3)
Coordinates:
  * sizeBin                  (sizeBin) int64 0 1
  * sizeBin1                 (sizeBin1) int64 0 1 2
  * layer                    (layer) int64 0 1 2
  * frequency                (frequency) float64 3e+09
Data variables:
    sizeBounds               (sizeBin1) float64 0.001 0.001 0.001
    sizeCenter               (sizeBin) float64 0.001 0.001
    sizeBoundsWidth          (sizeBin) float64 5e-11 5e-11
    aspectRatio              float64 1.0
    density                  float64 1e+03
    mass                     (sizeBin) float64 5.236e-07 5.236e-07
    crossSectionArea         (sizeBin) float64 7.854e-07 7.854e-07
    sizeDistribution         (layer, sizeBin) float64 1e+08 1e+08 1e+10 ...
    relativePermittivity     (layer, frequency) complex128 dask.array<shape=(3, 1), chunksize=(1, 1)>
    fallVelocity             (sizeBin, layer) float64 dask.array<shape=(2, 3), chunksize=(

# Radar simulator


In [10]:
pam2.profile['pathIntegratedAtenuattion'] = xr.zeros_like(pam2.hydrometeors.cloud.profile.backscatterCrossSection.isel(sizeBin=0))

pam2.addInstrument(
    pamtra2.instruments.radar.simpleRadar,
    name = 'simple',
    frequencies = 3e9,    
        radarK2=0.93,    
)

In [11]:
pam2.addInstrument(
    pamtra2.instruments.radar.dopplerRadarPamtra,
    name = 'spectral',
    frequencies = 3e9,    
        verbosity=10,
    radarNAve =150,
    momentsNPeaks = 1,
    seed = 11,
    radarAliasingNyquistInterv = 0,
    radarPNoise1000 = -100
)



In [12]:

pam2.instruments.simple.results.compute()

<xarray.Dataset>
Dimensions:            (frequency: 1, layer: 3)
Coordinates:
  * layer              (layer) int64 0 1 2
  * frequency          (frequency) float64 3e+09
    sizeBin            int64 0
Data variables:
    radarReflectivity  (frequency, layer) float64 -19.97 0.03176 20.03

In [13]:
pam2.instruments.spectral.results.compute()


<xarray.Dataset>
Dimensions:                 (dopplerVelocity: 256, dopplerVelocityAliased: 256, frequency: 1, layer: 3, peak: 1)
Coordinates:
  * layer                   (layer) int64 0 1 2
  * frequency               (frequency) float64 3e+09
    sizeBin                 int64 0
  * dopplerVelocity         (dopplerVelocity) float64 -7.885 -7.823 -7.762 ...
  * peak                    (peak) int64 1
Dimensions without coordinates: dopplerVelocityAliased
Data variables:
    radarIdealizedSpectrum  (dopplerVelocityAliased, layer, frequency) float64 0.0 ...
    radarSpectrum           (dopplerVelocity, layer, frequency) float64 6.217e-12 ...
    radarReflectivity       (peak, layer, frequency) float64 -19.91 0.1033 20.02
    meanDopplerVel          (peak, layer, frequency) float64 3.693 3.702 3.686
    spectrumWidth           (peak, layer, frequency) float64 0.3954 0.3957 ...
    skewness                (peak, layer, frequency) float64 -0.01364 ...
    kurtosis                (peak, layer